In [1]:
# imports
from pathlib import Path
import numpy as np
from scipy.signal import resample
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Flatten, Convolution1D, Dropout, BatchNormalization, MaxPooling1D, AveragePooling1D
from keras import regularizers
from keras import initializers
from keras.optimizers import Adam, SGD, Adagrad
from keras.utils import np_utils
from keras.callbacks import TensorBoard

Using TensorFlow backend.


## try fully connceted networks.

In [2]:
## data loading
## data loading
datapath = Path('../dataset')
xfile = 'X_features_spec.npy'
yfile = 'Y_labels_spec.npy'
def load_waveforms():
    X_list = np.load(str(datapath.joinpath(xfile)))
    Y_list = np.load(str(datapath.joinpath(yfile)))
    return X_list, Y_list

def positve_samples(xlist):
    ## some samples have negative signs
    xl_new = []
    for sample in range(xlist.shape[0]):
        points = xlist[sample]
        for p in range(points.shape[0]):
            point = points[p]
            if np.sum(point) < 0:
                points[p] = -point
        xl_new.append(points)
    return np.array(xl_new)

## apply to loaded dataset
def split_by_channel(xlist):
    ## input as (n, 2500)
    def standard_resample(arr):
        return resample(arr, 2500)
    ## if some is not with dim 625, resample it
    xl_new = []
    for sample in range(xlist.shape[0]):
        points = xlist[sample]
        if points.shape[1] != 2500:
            print("resample")
            print(points.shape)
            points = np.apply_along_axis(standard_resample, axis=1, arr=points)
        points = points.reshape((points.shape[0], 625, 4))
        xl_new.append(points)
    return np.array(xl_new)

## input is after split
def apply_resample(xlist, outdim):
    ## resample
    def resample_waveform(arr):
        ## arr.shape: (indim, )
        return resample(arr, outdim)
    xl_new = []
    for sample in range(xlist.shape[0]):
        points = xlist[sample]
        points = np.apply_along_axis(resample_waveform, axis=1, arr=points)
        xl_new.append(points)
    return np.array(xl_new)

## input is combined exp. (18000 ,625, 4)
def get_xtrain_mean(x_train):
    ## mean value for each dimension (exp. each of 625 dim)
    m = np.mean(x_train, axis=0)
    ## then we can apply x_train - m for zero mean
    return m

## input is after split
## one variance for each channel
def normalize_waveform():
    ## we don't necessarily need this
    pass

def combine_samples(arrs):
    ## exp. arrs.shape: (20, ?)
    pass

def binary_label(ylist):
    ## 1, 2 --> 1
    ylist_new = []
    for sample in range(ylist.shape[0]):
        labels = ylist[sample]
        labels[labels > 1] = 1
        ylist_new.append(labels)
    return np.array(ylist_new)

def combine_samples(arrs):
    ## exp. arrs.shape: (20, ?)
    if arrs.shape[0] < 1:
        return arrs
    sp = list(arrs[0].shape)
    sp[0] = 0
    combined = np.zeros(sp)
    print("combinde", combined.shape)
    for sample in range(arrs.shape[0]):
        arr = arrs[sample]
        combined = np.concatenate((combined, arr), axis=0)
    return combined

In [3]:
## global parameters

In [4]:
## one time parameter for the model below
## regularizer
## l2
ker_reg = 0.1
act_reg = 0.1
## kernel_initializer
ker_init = initializers.glorot_normal(seed=None)
## shape
in_shape = (648, 4)
## learning rate
opt = Adam()
opt.lr = 0.0001
##
OUTPUT_SIZE = 2
##
epochs = 20
## callback
model_callback = TensorBoard(log_dir='./logs', histogram_freq=0, write_graph=True, write_images=False)

this time, add a conv layer with bigger kernel size and stride at first --> more generalized?

In [5]:
## model
## resample data to 640 * 4
model = Sequential()
model.add(Dense(500, activation='relu', input_shape=in_shape))
model.add(Dense(300, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Flatten())
## fully connected
model.add(Dense(OUTPUT_SIZE))
## softmax
model.add(Activation('softmax'))

In [6]:
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 648, 500)          2500      
_________________________________________________________________
dense_2 (Dense)              (None, 648, 300)          150300    
_________________________________________________________________
dense_3 (Dense)              (None, 648, 100)          30100     
_________________________________________________________________
flatten_1 (Flatten)          (None, 64800)             0         
_________________________________________________________________
dense_4 (Dense)              (None, 2)                 129602    
_________________________________________________________________
activation_1 (Activation)    (None, 2)                 0         
Total params: 312,502
Trainable params: 312,502
Non-trainable params: 0
_________________________________________________________________
None

In [7]:
## experiment
## case 12, 14 for test
## case 18 for validation
## other case for traning
x_list, y_list = load_waveforms()
x_list = positve_samples(x_list)
x_list = split_by_channel(x_list)
x_list = apply_resample(x_list, 648)
y_list = binary_label(y_list)
for i in range(x_list.shape[0]):
    print(x_list[i].shape)

resample
(2490, 1920)
resample
(878, 1920)
(2394, 648, 4)
(4144, 648, 4)
(3302, 648, 4)
(1272, 648, 4)
(389, 648, 4)
(2716, 648, 4)
(61, 648, 4)
(628, 648, 4)
(611, 648, 4)
(771, 648, 4)
(201, 648, 4)
(1354, 648, 4)
(2490, 648, 4)
(878, 648, 4)
(2506, 648, 4)
(1688, 648, 4)
(2067, 648, 4)
(1554, 648, 4)
(635, 648, 4)
(1439, 648, 4)


In [8]:
val_idx = [17]
test_idx = [11, 13]
train_list_x = []
train_list_y = []
val_list_x = []
val_list_y = []
test_list_x = []
test_list_y = []
for idx in range(x_list.shape[0]):
    if idx not in (val_idx + test_idx):
        train_list_x.append(x_list[idx])
        train_list_y.append(y_list[idx])
        
for idx in val_idx:
    val_list_x.append(x_list[idx])
    val_list_y.append(y_list[idx])
    
for idx in test_idx:
    test_list_x.append(x_list[idx])
    test_list_y.append(y_list[idx])  

train_list_x = np.array(train_list_x)
train_list_y = np.array(train_list_y)
val_list_x = np.array(val_list_x)
val_list_y = np.array(val_list_y)
test_list_x = np.array(test_list_x)
test_list_y = np.array(test_list_y)
train_list_x = combine_samples(train_list_x)
train_list_y = combine_samples(train_list_y)
val_list_x = combine_samples(val_list_x)
val_list_y = combine_samples(val_list_y)
test_list_x = combine_samples(test_list_x)
test_list_y = combine_samples(test_list_y)
train_list_y = np_utils.to_categorical(train_list_y, num_classes=2)
val_list_y = np_utils.to_categorical(val_list_y, num_classes=2)
test_list_y = np_utils.to_categorical(test_list_y, num_classes=2)

combinde (0, 648, 4)
combinde (0,)
combinde (0, 648, 4)
combinde (0,)
combinde (0, 648, 4)
combinde (0,)


In [9]:
model.fit(train_list_x, train_list_y,
          epochs=epochs,
          verbose=2,
          validation_data=(val_list_x, val_list_y),
          callbacks=[model_callback])

Train on 27314 samples, validate on 1554 samples
Epoch 1/20
 - 13s - loss: 0.2405 - acc: 0.9131 - val_loss: 0.6429 - val_acc: 0.7625
Epoch 2/20
 - 13s - loss: 0.1615 - acc: 0.9441 - val_loss: 0.5528 - val_acc: 0.7767
Epoch 3/20
 - 13s - loss: 0.1508 - acc: 0.9461 - val_loss: 0.5885 - val_acc: 0.7806
Epoch 4/20
 - 13s - loss: 0.1431 - acc: 0.9501 - val_loss: 0.8082 - val_acc: 0.7381
Epoch 5/20
 - 13s - loss: 0.1391 - acc: 0.9507 - val_loss: 0.5637 - val_acc: 0.7915
Epoch 6/20
 - 13s - loss: 0.1370 - acc: 0.9502 - val_loss: 0.5967 - val_acc: 0.7696
Epoch 7/20
 - 13s - loss: 0.1329 - acc: 0.9524 - val_loss: 0.6618 - val_acc: 0.7548
Epoch 8/20
 - 13s - loss: 0.1321 - acc: 0.9519 - val_loss: 0.6056 - val_acc: 0.7934
Epoch 9/20
 - 13s - loss: 0.1291 - acc: 0.9531 - val_loss: 0.7676 - val_acc: 0.7297
Epoch 10/20
 - 13s - loss: 0.1247 - acc: 0.9546 - val_loss: 0.7528 - val_acc: 0.7400
Epoch 11/20
 - 13s - loss: 0.1263 - acc: 0.9542 - val_loss: 0.6783 - val_acc: 0.7735
Epoch 12/20
 - 13s - loss

In [11]:
loss, acc = model.evaluate(test_list_x, test_list_y)
print("loss", loss, "acc", acc)

2232/2232 [==============================] - 0s 213us/step
loss 2.242920946903981 acc 0.3068996416304701
